In [2]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import re
import os

urls = [
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S01.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S02.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S03.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S04.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S06.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S07.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S08.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S09.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S10.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S11.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S12.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S13.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S14.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S15.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S16.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S17.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S18.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S19.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-S20.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U01.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U02.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U03.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U04.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U05.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U06.htm",
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U07.htm"
    "https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-U08.htm"
]

def extract_state_name(url):
    return url.split('-')[-1].split('.')[0]

def process_url(url):
    state_name = extract_state_name(url)  
    directory = f'./{state_name}'
    if not os.path.exists(directory):
        os.makedirs(directory)   
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    party_seats = []
    grid_boxes = soup.find_all("div", class_="grid-box")
    for box in grid_boxes:
        party_name = box.find("h4").text
        seats_won = box.find("h2").text
        party_seats.append({"Party": party_name, "Seats": int(seats_won)})
    
    script = soup.find("script", string=lambda text: text and "var xValues" in text).string

    x_values_pattern = re.compile(r"var xValues = \[(.*?)\];", re.DOTALL)
    y_values_pattern = re.compile(r"var yValues = \[(.*?)\];", re.DOTALL)

    x_values_match = x_values_pattern.search(script)
    y_values_match = y_values_pattern.search(script)

    x_values = x_values_match.group(1).split(',')
    y_values = y_values_match.group(1).split(',')

    x_values = [x.strip().strip("'").strip('"') for x in x_values]
    y_values = [y.strip() for y in y_values]

    x_values = [x for x in x_values if x]
    y_values = [int(y) for y in y_values if y.isdigit()]

    party_vote_share = []
    for i in range(len(x_values)):
        party_vote_share.append({"Party": x_values[i], "Votes": y_values[i]})
    df_seats = pd.DataFrame(party_seats)
    df_vote_share = pd.DataFrame(party_vote_share)
    seat_plot_path = os.path.join(directory, f'{state_name}_party_seat_distribution.png')
    vote_share_plot_path = os.path.join(directory, f'{state_name}_party_vote_share.png')
    df_seats.plot(kind="bar", x="Party", y="Seats", title=f"Party-wise Seat Distribution ({state_name})", legend=False)
    plt.ylabel("Seats Won")
    plt.savefig(seat_plot_path)
    plt.close()
    df_vote_share.plot(kind="pie", y="Votes", labels=df_vote_share["Party"], autopct='%1.1f%%', title=f"Party-wise Vote Share ({state_name})")
    plt.ylabel("")
    plt.savefig(vote_share_plot_path)
    plt.close()

    csv_file_path = os.path.join(directory, f'{state_name}_election_results.csv')
    with open(csv_file_path, 'w') as file:
        file.write(f"Party-wise Seat Distribution ({state_name}):\n")
        df_seats.to_csv(file, index=False)
        file.write("\nPlot: " + seat_plot_path + "\n\n")    
        file.write(f"Party-wise Vote Share ({state_name}):\n")
        df_vote_share.to_csv(file, index=False)
        file.write("\nPlot: " + vote_share_plot_path + "\n\n")
    print(f"Data and plots saved for {state_name}")

for url in urls:
    process_url(url)


Data and plots saved for S01
Data and plots saved for S02
Data and plots saved for S03
Data and plots saved for S04
Data and plots saved for S05
Data and plots saved for S06
Data and plots saved for S07
Data and plots saved for S08


AttributeError: 'NoneType' object has no attribute 'string'